In [ ]:
import sys
!conda install -c conda --yes --prefix {sys.prefix} tensorflow

In [2]:
from glob import glob
import datetime as dt
import pandas as pd
import geopandas as gpd
from datetime import datetime
import keras

# Initialize Data

In [3]:
# Files with data
paths = glob('C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/sentinel_ismn_data/*',)
paths.sort()

# Create Station id linked with filepath
ids = [x.split('\\')[-1].split('_')[0] for x in paths]
files = dict(zip(ids,paths))

# Load all files into one Dataframe
gdf = gpd.tools.util.pd.concat(map(gpd.read_file, paths), ignore_index=True)

# Preprocessing

In [4]:
# Set type of date column to datetime object
gdf.date = gdf.date.astype('datetime64[ns]')

# Convert int to timedelta in days
s2_timedelta = [dt.timedelta(days=x) for x in gdf.s2_distance]
gdf.s2_distance = s2_timedelta

#Remove uneseccary columns
gdf.drop(labels = ['CloudMask'], axis = 1, inplace = True)

# Clean Nan Values within subset columns (major variables to inspect)
gdf.dropna(how='any', subset=['soil_moisture', 'VV'], inplace=True)

# Drop lower and upper 1% of data to eliminate outliers
gdf = gdf[gdf.soil_moisture.gt(gdf.soil_moisture.quantile(0.01)) & gdf.soil_moisture.lt(gdf.soil_moisture.quantile(0.99))]
gdf = gdf[gdf.VV.gt(gdf.VV.quantile(0.01)) & gdf.VV.lt(gdf.VV.quantile(0.99))]
#gdf = gdf[gdf.NDVI.gt(gdf.NDVI.quantile(0.01)) & gdf.NDVI.lt(gdf.NDVI.quantile(0.99))]

# Remove rows where ndvi is older than 30days
gdf = gdf[gdf.s2_distance.gt(dt.timedelta(days=-30)) & gdf.s2_distance.lt(dt.timedelta(days=30))]

# Select only sm and vv meassurements where soil is not in frozen state


## Scale, Transform dataset according to regression

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler

In [ ]:
https://www.pyimagesearch.com/2019/01/21/regression-with-keras/

# Create Training and Validation datasets

## Scaling

In [37]:
csc = MinMaxScaler()
lsc = LabelBinarizer()

## CNN Architecture

### MLP Multilayer Perceptron

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [4]:
def baseline_model(dim):
    # Create Model
    model = Sequential()
    model.add(Dense(dim, input_dim = dim, kernel_initializer = 'normal', activation = 'relu')) #hiddenlayer?
    model.add(Dense(1, kernel_initializer = 'normal')) #output layer?

    # Compile model
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    return model

In [60]:
def create_mlp(dim, regress=False):
    # MLP Model
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation='relu'))
    model.add(Dense(4, activation='relu'))
    
    # Check to see if regression node should be added
    if regress:
        model.add(Dense(1, activation='linear'))
    
    return model

## First Try: 2 Feature [sm, vv] / No Seperation between different Paramters 

In [9]:
X = gdf['VV'].reset_index(drop=True)
Y = gdf['soil_moisture'].reset_index(drop=True)

In [13]:
# Evaluate Model
estimator = KerasRegressor(
    build_fn = baseline_model(dim = 1),
    epochs = 100, 
    batch_size = 5,
    verbose = 0)
kfold = KFold(n_splits = 10) #10-fold cross validation to evaluate the model
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))
print(f"Baseline: {results.mean()}.2f ({results.std()}.2f) MSE")

Baseline: nan (nan) MSE
Baseline: nan.2f (nan.2f) MSE


C:\Users\USER\Anaconda3\envs\irrigation_detection\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\USER\Anaconda3\envs\irrigation_detection\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\USER\Anaconda3\envs\irrigation_detection\lib\site-packages\keras\wrappers\scikit_learn.py", line 150, in fit
    self.model = self.build_fn(
  File "C:\Users\USER\Anaconda3\envs\irrigation_detection\lib\site-packages\keras\engine\base_layer.py

In [71]:
features = ['VV', 'soil_moisture']
df = gdf[features].reset_index(drop=True)
sc_data = csc.fit_transform(df)
X_train, X_test, y_train, y_test = train_test_split(sc_data[:,0],sc_data[:,1], test_size=0.25, random_state = 42, shuffle=False, stratify=None)

In [72]:
model = models.create_mlp(X_train, regress=True)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss='mean_absolute_percentage_error', optimizer=opt)

# Train the model
model.fit(
    x = X_train,
    y = y_train,
    validation_data = (X_test, y_test),
    epoches = 200,
    batch_size = 8
    )

NameError: name 'models' is not defined

In [27]:
def prep_scaling(df, modus, features):
    """
    Arguments: modus ['MinMax', 'LabelBinarizer']
    """
    # Import modlues
    from sklearn.preprocessing import LabelBinarizer
    from sklearn.preprocessing import MinMaxScaler
    
    # Initialize Scaler ?What is with negative numbers?
    if modus == 'MinMax':
        cs = MinMaxScaler()
    elif modus == 'LabelBinarizer':
        cs = LabelBinarizer()
    
    return cs.fit_transform(df[columns])

In [ ]:
data_1 = prep_scaling(
    df = gdf,
    modus = 'MinMax',
    columns = ['VV', 'soil_moisture'],
    )
data_1 sklearn.model_selection.train_test_split

In [25]:
df_1 = gdf[['soil_moisture', 'VV']].reset_index(drop=True) #2 Feature Selection


x = gdf.VV.reset_index(drop=True) # Sentinel 1 GRD VV Backscatter Coefficient
y = gdf.soil_moisture.reset_index(drop=True) # Soil Moisture



,soil_moisture,VV
0,0.224,-6.456684
1,0.327,-9.103003
2,0.367,-8.022560
3,0.337,-6.844326
4,0.340,-7.574737
...,...,...
31609,0.340,-7.808508
31610,0.360,-9.553531
31611,0.338,-7.201390
31612,0.326,-11.959831


In [7]:
data = gpd.read_file(ismn_path + '\ismn_ts_vv_sm_01.json')
data['date'] = pd.to_datetime(data.date)

In [ ]:
ms1_x, ms1_y, ms2_x, ms2_y, as1_x, as1_y, as2_x, as2_y = list(), list(),list(),list(),list(),list(),list(),list()

for station in data.station.unique():
    for year in data.date.dt.year.unique():
        time_series = data[(data['station'] == station ) & (data['date'].dt.year == year) & (data['date'].dt.month.isin([3,4,5,6,7,8,9]))]
        #Sentinel A Ascending 
        ms1_x.append(time_series[(time_series['sentinel'] == 'A') & (time_series['orbit_direction'] == 'ASCENDING')]['VV'])
        ms1_y.append(time_series[(time_series['sentinel'] == 'A') & (time_series['orbit_direction'] == 'ASCENDING')][['angle', 'soil_moisture']])
        # Sentinel B Ascending
        ms2_x.append(time_series[(time_series['sentinel'] == 'B') & (time_series['orbit_direction'] == 'ASCENDING')]['VV'])
        ms2_y.append(time_series[(time_series['sentinel'] == 'B') & (time_series['orbit_direction'] == 'ASCENDING')][['angle', 'soil_moisture']])
        # Sentinel A Descending
        as1_x.append(time_series[(time_series['sentinel'] == 'A') & (time_series['orbit_direction'] == 'DESCENDING')]['VV'])
        as1_y.append(time_series[(time_series['sentinel'] == 'A') & (time_series['orbit_direction'] == 'DESCENDING')][['angle', 'soil_moisture']])
        # Sentinel B Descending
        as2_x.append(time_series[(time_series['sentinel'] == 'B') & (time_series['orbit_direction'] == 'DESCENDING')]['VV'])
        as2_y.append(time_series[(time_series['sentinel'] == 'B') & (time_series['orbit_direction'] == 'DESCENDING')][['angle', 'soil_moisture']])


## Download Sentinel-1 Metadata

In [ ]:
def get_s1_metadata(filename):
    from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
    # connect to the API
    api = SentinelAPI('till90', '201184Till', 'https://scihub.copernicus.eu/dhus')
    # Query Metadata by filename
    products = api.query(filename = f'{filename}*')
    return api.to_dataframe(products)

In [ ]:
get_s1_metadata('S1A_IW_GRDH_1SDV_20150104T051921_20150104T051946_004017_004D70_3611').orbitdirection.values[0]

## Data to train the CNN

### ismn in-situ soil moisture

In [6]:
#path to single json files per station
ismn_path = r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\ground_trouth\ismn_archieve\time_series'
files = glob(ismn_path + '\*')

In [ ]:
files = files[:-1]

In [ ]:
#load time series in pandas
ts_ismn_sm = [pd.read_json(x) for x in files]
# Get coords from filename
coords = [x.split('\\')[-1].split('_')[1:3] for x in files]
# Create GeoDataFrame for every time series with geometry from filename 
ts_ismn_sm_gdf = [gpd.GeoDataFrame(data = ts_ismn_sm[x], geometry=gpd.points_from_xy(x = np.repeat(coords[x][1], len(ts_ismn_sm[x])), y = np.repeat(coords[x][0], len(ts_ismn_sm[x])))) for x in range(0,len(files))]
# Concat all gdf to one geodataframe
ts_ismn = gpd.tools.util.pd.concat(ts_ismn_sm_gdf)

In [ ]:
coords

In [ ]:
# Extract regular sampled; 1 values for 1 day at the same time s1 meassure backscatter coeffienct


### gee sentinel 1 data

In [ ]:
#path to single json files per station
gee_fc_path = r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\ground_trouth\ismn_fc_gee_VV_na_clear.geojson'
ts_ismn_bc = gpd.read_file(gee_fc_path)

In [ ]:
# Extract metadata from filename
ts_ismn_bc['station'] = [x[-1] for x in ts_ismn_bc.id]
ts_ismn_bc['sentinel'] = [x[2] for x in ts_ismn_bc.id]
ts_ismn_bc['date'] = [datetime.strptime(x.split('_')[4][:15], '%Y%m%dT%H%M%S') for x in ts_ismn_bc.id]
ts_ismn_bc['mode'] = [x[4:6] for x in ts_ismn_bc.id]
ts_ismn_bc['type'] = [str(x[7:11]) for x in ts_ismn_bc.id]
# only High Resolution types
ts_ismn_bc = ts_ismn_bc[ts_ismn_bc['type'] == 'GRDH']
#drop duplicates fix because when downloading vv data some sensors are at same postion but different depths
ts_ismn_bc.drop_duplicates(['geometry', 'VV'], inplace=True)
# Add Orbit Direction with sentinelsat query
orbit_directions = list()
for row in ts_ismn_bc.itertuples():
    metadata = get_s1_metadata(row.id[:-2])
    try:
        orbit_directions.append(metadata.orbitdirection.values[0])
    except:
        orbit_directions.append(np.nan)
ts_ismn_bc['orbit_direction'] = orbit_directions
ts_ismn_bc

## Merge data vv + sm ground trouth

In [ ]:
# Add sm from ismn data to s1 vv data
values = list()
for row in ts_ismn_bc.itertuples():
    time = row.date.round('1h')
    geometry = row.geometry
    # select all rows with same geometry (change x and y lat/lon maybe because of shaply module)
    sm = ts_ismn.cx[geometry.y:geometry.y , geometry.x:geometry.x]
    # select all rows with same time
    sm = sm[sm.index == time]
    # mean when multiple values nan when no values
    if len(sm) == 0:
        values.append(np.nan)
    else:
        values.append(np.mean(list(sm.soil_moisture)))
        
ts_ismn_bc['soil_moisture'] = values
ts_ismn_bc.to_file(ismn_path + '\ismn_ts_vv_sm_01.json')

In [ ]:
ts_ismn_bc.station.unique()